In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Libraries
import os
import gc
import shutil
import cv2
import numpy as np 
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

 #Data Visualization
from glob import glob
import xml.etree.ElementTree as xet
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image, display

In [ ]:
import xml.etree.ElementTree as ET  # or use your alias, e.g., import xml.etree.ElementTree as xet
from glob import glob

def append_labels_from_xml(glob_path, labels_dict):
    """
    Processes XML files from a given glob path and appends data to the labels_dict.
    
    Parameters:
        glob_path (str): The glob pattern to find XML files.
        labels_dict (dict): A dictionary with keys 'filepath', 'xmin', 'xmax', 'ymin', 'ymax', 'plate_number'.
                            Data from each XML file will be appended to these lists.
    """
    # Get a list of files matching the glob pattern
    file_list = glob(glob_path)
    #print(f"Processing {len(file_list)} files from: {glob_path}")
    
    for filename in file_list:
        tree = ET.parse(filename)
        root = tree.getroot()
        #print("Path is:", root.find('path').text)
        
        # Extract the necessary information
        obj = root.find('object')
        plate_number = obj.find('name').text.strip()
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        
        # Append the extracted data to the provided dictionary
        labels_dict['filepath'].append(filename)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)
        labels_dict['plate_number'].append(plate_number)

# Example usage:

# Initialize your dictionary (this can later be converted to a DataFrame)
new_labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[], plate_number=[])

new_test_labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[], plate_number=[])

new_labels_video_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[], plate_number=[])
 
# Process files from one glob path
append_labels_from_xml('/kaggle/input/indian-vehicle-dataset/State-wise_OLX/*/*.xml', new_labels_dict)
append_labels_from_xml('/kaggle/input/indian-vehicle-dataset/video_images/*.xml', new_labels_video_dict)
append_labels_from_xml('/kaggle/input/indian-vehicle-dataset/google_images/*.xml', new_test_labels_dict)

# Now, new_labels_dict contains appended data from all the XML files processed.


In [ ]:
df_olx = pd.DataFrame(new_labels_dict)
df_olx.to_csv('labels_train.csv',index=False)
print(df_olx.head())
df_olx.count()

df_google = pd.DataFrame(new_test_labels_dict)
df_google.to_csv('labels_test.csv',index=False)
print(df_google.head())
df_google.count()

df_video = pd.DataFrame(new_labels_video_dict)
df_video.to_csv('labels_train.csv',index=False)
print(df_video.head())
df_video.count()



In [ ]:
# parsing
def addcolumns_train(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/indian-vehicle-dataset/State-wise_OLX/{name[:2]}/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df_olx[['filename','width','height']] = df_olx['filepath'].apply(addcolumns_train).apply(pd.Series)
df_olx.head()
df_olx.count()


# parsing
def addcolumns_test(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/indian-vehicle-dataset/google_images/{name}'
    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df_google[['filename','width','height']] = df_google['filepath'].apply(addcolumns_test).apply(pd.Series)
df_google.head()
df_google.count()


# parsing
def addcolumns_video_train(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/indian-vehicle-dataset/video_images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df_video[['filename','width','height']] = df_video['filepath'].apply(addcolumns_video_train).apply(pd.Series)
df_video.head()
df_video.count()



In [ ]:
# # center_x, center_y, width , height
# df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
# df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

# df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
# df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
# df.head()

In [ ]:
# df = df.rename(columns={"filename": "image_path"})
# # Extract individual bbox coordinates
# df["x1_bbox"] = df["xmin"] 
# df["y1_bbox"] = df["ymin"] 
# df["x2_bbox"] = df["xmax"] 
# df["y2_bbox"] = df["ymax"]
# df["xmid"] = df["center_x"]
# df["ymid"] = df["center_y"]
# df = df.drop(columns=["filepath", "xmin", "xmax", "ymin", "ymax", "width", "height", "center_x", "center_y"])
# df.count()

In [ ]:
def process_dataframe(df):
    """
    Modifies the given DataFrame in place by performing the following operations:
    
    1. Computes 'center_x' and 'center_y' columns as:
         center_x = (xmax + xmin) / (2 * width)
         center_y = (ymax + ymin) / (2 * height)
    
    2. Computes bounding box dimensions 'bb_width' and 'bb_height' as:
         bb_width  = (xmax - xmin) / width
         bb_height = (ymax - ymin) / height
    
    3. Renames the column 'filename' to 'image_path'.
    
    4. Creates new columns for individual bounding box coordinates:
         - x1_bbox from xmin
         - y1_bbox from ymin
         - x2_bbox from xmax
         - y2_bbox from ymax
         - xmid from center_x
         - ymid from center_y
    
    5. Drops the unnecessary columns:
         ['filepath', 'xmin', 'xmax', 'ymin', 'ymax', 'width', 'height', 'center_x', 'center_y']
    """
    # Compute center_x and center_y
    df['center_x'] = (df['xmax'] + df['xmin']) / (2 * df['width'])
    df['center_y'] = (df['ymax'] + df['ymin']) / (2 * df['height'])
    
    # Compute bounding box width and height
    df['bb_width'] = (df['xmax'] - df['xmin']) / df['width']
    df['bb_height'] = (df['ymax'] - df['ymin']) / df['height']
    
    # Rename the column 'filename' to 'image_path' in place
    df.rename(columns={"filename": "image_path"}, inplace=True)
    
    # Extract individual bbox coordinates and midpoints
    df["x1_bbox"] = df["xmin"]
    df["y1_bbox"] = df["ymin"]
    df["x2_bbox"] = df["xmax"]
    df["y2_bbox"] = df["ymax"]
    df["xmid"] = df["center_x"]
    df["ymid"] = df["center_y"]
    
    # Drop the unnecessary columns
    columns_to_drop = ["filepath", "xmin", "xmax", "ymin", "ymax", "width", "height", "center_x", "center_y"]
    df.drop(columns=columns_to_drop, inplace=True)



In [ ]:
print("Processing olx train data")
process_dataframe(df_olx)
print(df_olx.head())
print("Processing google test data")
process_dataframe(df_google)
print(df_google.head())

print("Processing video train data")
process_dataframe(df_video)
print(df_video.head())

In [ ]:
df_train_olx_video = pd.concat([df_olx,df_video]).sample(frac=1).reset_index(drop=True)
df_train_olx_video.head()

In [ ]:
df_train_olx_video.count()

In [ ]:
import cv2
from sklearn.model_selection import train_test_split



# Split the data into train and test sets
train_df, test_df = train_test_split(df_train_olx_video, test_size=0.2, random_state=42)

# Create directories for YOLO dataset
os.makedirs("/kaggle/working/yolo_dataset/train/images", exist_ok=True)
os.makedirs("/kaggle/working/yolo_dataset/train/labels", exist_ok=True)
os.makedirs("/kaggle/working/yolo_dataset/test/images", exist_ok=True)
os.makedirs("/kaggle/working/yolo_dataset/test/labels", exist_ok=True)

# Convert dataset format to YOLO
def convert_to_yolo(row, img_width, img_height):
    x_center = ((row['x1_bbox'] + row['x2_bbox']) / 2) / img_width
    y_center = ((row['y1_bbox'] + row['y2_bbox']) / 2) / img_height
    width = row['bb_width']
    height = row['bb_height']
    return f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"


def create_yolo_dataset(is_train,df):

    if is_train:
        folder_type = 'train'
    else:
        folder_type = 'test'

    print("folder type ",folder_type)
    skipped_images = 0
    
    # Process images and labels
    
#   for index, row in tqdm(df.iterrows(), total=len(df)):
    for index, row in df.iterrows():
        img_path = row['image_path']
        #print(f"Trying to read: {img_path}")  # Debugging step
        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠ Warning: Could not load image {img_path}..skipping")
            skipped_images += 1
            continue
    
        h, w, _ = img.shape  # Get image size
    
        # Convert bbox to YOLO format
        yolo_annotation = convert_to_yolo(row, img_width=w, img_height=h)
    
        # Save image
        img_filename = f"/kaggle/working/yolo_dataset/{folder_type}/images/{index}.jpg"
        cv2.imwrite(img_filename, img)
    
        # Save label
        label_filename = f"/kaggle/working/yolo_dataset/{folder_type}/labels/{index}.txt"
        with open(label_filename, "w") as f:
            f.write(yolo_annotation)
    print("skipped images", skipped_images)

create_yolo_dataset(True,train_df)
create_yolo_dataset(False,test_df)

print("✅ Data conversion complete. Ready for YOLO training.")

In [ ]:
train_df.count()
test_df.count()

In [ ]:
# Create a data.yaml file with the correct paths

import yaml

data_dict = {
    'train': '/kaggle/working/yolo_dataset/train/images',
    'val': '/kaggle/working/yolo_dataset/test/images',
    'nc': 1,  # Update based on the number of classes in your dataset
    'names': ['license_plate']  # Update based on your class names
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_dict, f)

print("data.yaml file created successfully!")

In [ ]:
!pip install ultralytics


import ultralytics
from ultralytics import YOLO

# Train YOLO on the dataset
model = YOLO("yolov8n.pt")  # Use a pre-trained model



# Train YOLO with custom dataset
model.train(data="data.yaml", epochs=50, imgsz=640)

# Save the model to a specific path
model.save('/kaggle/working/mymodel.pt')

# Load YOLO model

#model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")  # Path to trained model
model = YOLO('/kaggle/working/mymodel.pt')
print("Not using Yolov8N")
print(model)


In [ ]:
# import re

# def extract_license_plate(ocr_result, confidence_threshold=0.5, size_threshold_factor=0.5):
#     # If OCR result is empty or improperly formatted, return a default value
#     if not ocr_result or not ocr_result[0]:
#         return "No license plate detected"
    
#     # Initialize a list to hold texts that pass filtering
#     detected_texts = []
#     max_area = 0  # To track the largest detected text area

#     # Loop through OCR detections
#     for detection in ocr_result[0]:  # Extract detections
#         if len(detection) > 0:
#             text = detection[1][0]  # Extract the text
#             confidence = detection[1][1]  # Extract confidence score
#             box = detection[0]  # Bounding box (coordinates)
            
#             # Calculate the area of the bounding box (width * height)
#             width = abs(box[1][0] - box[0][0])
#             height = abs(box[2][1] - box[1][1])
#             area = width * height

#             # Track maximum area for size comparison
#             max_area = max(max_area, area)

#             # Filter based on confidence score and size (compared to largest detected text area)
#             if confidence > confidence_threshold and area > (max_area * size_threshold_factor):
#                 detected_texts.append(text)

#     # Concatenate all filtered texts into one string
#     final_text = " ".join(detected_texts)
    
#     # Remove non-alphanumeric characters using regex
#     cleaned_text = re.sub(r'[^A-Za-z0-9]', '', final_text)
    
#     return cleaned_text

# # Example OCR result
# ocr_result = [[[[[12.0, 21.0], [161.0, 22.0], [160.0, 39.0], [12.0, 37.0]], ('MH 20BQ20', 0.8835029006004333)]], 
#                [[[[12.0, 41.0], [70.0, 40.0], [69.0, 60.0], [12.0, 61.0]], ('SMALL', 0.45)]]]

# # Extract license plate number with highest confidence
# license_plate_text = extract_license_plate(ocr_result)
# print("Extracted License Plate:", license_plate_text)


In [ ]:
import re
import xml.etree.ElementTree as ET
from glob import glob
import numpy as np

# Define a simple confusion mapping for common OCR errors
digit_to_letter = {'0': 'O', '1': 'I', '2': 'Z', '5': 'S', '6': 'G', '8': 'B', '9': 'P'}
letter_to_digit = {v: k for k, v in digit_to_letter.items()}

def extract_license_plate(ocr_result, confidence_threshold=0.5, size_threshold_factor=0.5):
    """
    Extracts and cleans text from OCR output, then applies a simple HMM correction
    based on the expected license plate format: AAXXAAXXXX (where A=alphabet, X=digit).
    """
    # If OCR result is empty or improperly formatted, return a default value
    if not ocr_result or not ocr_result[0]:
        return "No license plate detected"
    
    # Initialize a list to hold texts that pass filtering
    detected_texts = []
    max_area = 0  # To track the largest detected text area

    # Loop through OCR detections
    for detection in ocr_result[0]:  # Assuming detections are in the first element
        if len(detection) > 0:
            text = detection[1][0]          # Extract the text
            confidence = detection[1][1]      # Extract confidence score
            box = detection[0]                # Bounding box (coordinates)
            
            # Calculate the area of the bounding box (width * height)
            width = abs(box[1][0] - box[0][0])
            height = abs(box[2][1] - box[1][1])
            area = width * height

            # Update maximum area for size comparison
            max_area = max(max_area, area)

            # Filter based on confidence score and size (compared to largest detected text area)
            if confidence > confidence_threshold and area > (max_area * size_threshold_factor):
                detected_texts.append(text)

    # Concatenate all filtered texts into one string
    final_text = " ".join(detected_texts)
    
    # Remove non-alphanumeric characters using regex
    cleaned_text = re.sub(r'[^A-Za-z0-9]', '', final_text)
    
    # --- HMM Correction Step ---
    # We expect a license plate of format: AAXXAAXXXX 
    # (positions 0-1 and 4-5 are letters; positions 2-3 and 6-9 are digits)
    
    def hmm_correct_plate(obs_text):
        expected_types = ['letter', 'letter', 'digit', 'digit', 
                          'letter', 'letter', 'digit', 'digit', 'digit', 'digit']
        n = len(expected_types)
        
        # If the observed text is not 10 characters, you might want to handle it differently.
        # For now, we only correct if we have exactly 10 characters.
        if len(obs_text) != n:
            return obs_text  # Or return an error/default value
        
        # Candidate sets for each expected type
        letter_candidates = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
        digit_candidates = list("0123456789")
        
        # Define emission probability function.
        # We assume:
        #   - if observed char is of the expected type and matches candidate: 0.9
        #   - if observed char is of the expected type but not equal: small probability
        #   - if observed char is of the opposite type but can be mapped via confusion: moderate probability
        #   - otherwise, very low probability.
        def emission_prob(expected_type, candidate, observed):
            candidate = candidate.upper() if expected_type == 'letter' else candidate
            observed = observed.upper() if expected_type == 'letter' else observed
            if expected_type == 'letter':
                if observed.isalpha():
                    return 0.9 if candidate == observed else 0.1 / (len(letter_candidates)-1)
                elif observed.isdigit():
                    # Allow mapping from digit to letter if common OCR confusion exists
                    if digit_to_letter.get(observed, None) == candidate:
                        return 0.5
                    else:
                        return 0.01
                else:
                    return 0.01
            else:  # expected digit
                if observed.isdigit():
                    return 0.9 if candidate == observed else 0.1 / (len(digit_candidates)-1)
                elif observed.isalpha():
                    if letter_to_digit.get(observed, None) == candidate:
                        return 0.5
                    else:
                        return 0.01
                else:
                    return 0.01
        
        # For simplicity, we use uniform (or identity) transition probabilities.
        # Our state sequence is fixed by the format so each position is independent given the observation.
        # We still demonstrate a Viterbi algorithm.
        
        dp = []         # dp[i] will be a dict mapping candidate char at position i to probability
        backpointer = []  # To recover the best sequence
        
        # Initialization for position 0
        current_candidates = letter_candidates if expected_types[0] == 'letter' else digit_candidates
        dp0 = {}
        bp0 = {}
        for c in current_candidates:
            dp0[c] = emission_prob(expected_types[0], c, obs_text[0])
            bp0[c] = None
        dp.append(dp0)
        backpointer.append(bp0)
        
        # Recursion for positions 1 to n-1
        for i in range(1, n):
            current_candidates = letter_candidates if expected_types[i] == 'letter' else digit_candidates
            dp_curr = {}
            bp_curr = {}
            for curr in current_candidates:
                max_prob = -1
                best_prev = None
                # Since transition probabilities are uniform, we only multiply by emission probability.
                for prev, prev_prob in dp[i-1].items():
                    prob = prev_prob * emission_prob(expected_types[i], curr, obs_text[i])
                    if prob > max_prob:
                        max_prob = prob
                        best_prev = prev
                dp_curr[curr] = max_prob
                bp_curr[curr] = best_prev
            dp.append(dp_curr)
            backpointer.append(bp_curr)
        
        # Termination: pick the candidate at the last position with maximum probability.
        last_candidates = dp[-1]
        best_last = max(last_candidates, key=last_candidates.get)
        
        # Backtrace to retrieve the best candidate sequence.
        best_sequence = [best_last]
        for i in range(n-1, 0, -1):
            best_sequence.insert(0, backpointer[i][best_sequence[0]])
        
        return "".join(best_sequence)
    
    # Apply HMM correction if the cleaned text is 10 characters long;
    # otherwise, return the cleaned text as is.
    
    if len(cleaned_text) == 10:
        print("calling hmm corrct plate for",cleaned_text)
        corrected_text = hmm_correct_plate(cleaned_text)
        print("corrected text",corrected_text)

    else:
        print("directly assigned cleaned text")
        corrected_text = cleaned_text

    corrected_text = cleaned_text
    return corrected_text

# --- Example OCR result ---
ocr_result = [
    [  # First group of detections (we assume these are the license plate candidates)
        [
            [[12.0, 21.0], [161.0, 22.0], [160.0, 39.0], [12.0, 37.0]],
            ('A82BDD1963', 0.8835029006004333)
        ]
    ],
    [  # A second detection that might be filtered out (low confidence, etc.)
        [
            [[12.0, 41.0], [70.0, 40.0], [69.0, 60.0], [12.0, 61.0]],
            ('SMALL', 0.45)
        ]
    ]
]

# Extract and correct the license plate number
license_plate_text = extract_license_plate(ocr_result)
print("Extracted License Plate:", license_plate_text)


In [ ]:
import re

# Mapping common OCR misreads
digit_to_letter = {'0': 'O', '1': 'I', '2': 'Z', '5': 'S', '6': 'G', '8': 'B', '9': 'P'}
letter_to_digit = {v: k for k, v in digit_to_letter.items()}

def extract_license_plate_easyocr(ocr_result, confidence_threshold=0.5):
    """
    Extracts and cleans text from EasyOCR output.

    Args:
        ocr_result (list): Output from EasyOCR, formatted as [[bbox, text, confidence], ...].
        confidence_threshold (float): Minimum confidence to consider a detection.

    Returns:
        str: Extracted and corrected license plate text.
    """
    if not ocr_result:
        return "No license plate detected"
    
    detected_texts = []
    
    for detection in ocr_result:
        bbox, text, confidence = detection  # Unpack the detection
        
        if confidence > confidence_threshold:
            detected_texts.append(text)

    # Combine detected text parts
    final_text = " ".join(detected_texts)

    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^A-Za-z0-9]', '', final_text)

    return cleaned_text

# --- Example EasyOCR result ---
easyocr_result = [
    [[12, 21, 161, 22], 'A82BDD1963', 0.88],
    [[12, 41, 70, 40], 'SMALL', 0.45]  # This will be ignored due to low confidence
]

# Extract the license plate
license_plate_text = extract_license_plate_easyocr(easyocr_result)
print("Extracted License Plate:", license_plate_text)


In [ ]:
!pip install paddleocr
!pip install paddlepaddle


In [ ]:
!pip install python-Levenshtein


In [ ]:
# model = YOLO("/kaggle/input/apr/pytorch/default/1/ANPR_Best21.pt")  # Path to trained model
# print('model loaded')
# print(model)

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import cv2
import os
import easyocr
from difflib import SequenceMatcher
from tqdm import tqdm
from paddleocr import PaddleOCR
import logging
import Levenshtein

classes = ["Licence Plate"]

# List to store OCR results
detected_plates = []

# Accuracy counters
correct_predictions = 0
total_characters = 0
character_accuracy_sum = 0
levenshtein_accuracy_sum = 0

# Function to calculate character-level accuracy
def character_level_accuracy(ocr_text, ground_truth):
    # Remove any extra spaces
   # Clean the OCR text by joining parts and removing spaces
    ocr_text = ''.join(ocr_text).replace(" ", "").lower()
    print("final ocr_text",ocr_text)
    ground_truth = ground_truth.replace(" ", "").lower()
    
    correct_characters = 0
    invalid_images = 0
    total_characters = len(ground_truth)
    
    # Compare each character in OCR and ground truth
    for i in range(min(len(ocr_text), len(ground_truth))):
        if ocr_text[i] == ground_truth[i]:
            correct_characters += 1
    
    return correct_characters / total_characters if total_characters > 0 else 0

# Function to calculate word-level accuracy (Exact match)
def word_level_accuracy(ocr_text, ground_truth):
    ocr_text = ocr_text.upper()
    ground_truth = ground_truth.upper()
    return 1 if ocr_text.strip() == ground_truth.strip() else 0

# Function to calculate Levenshtein Distance-based accuracy
def levenshtein_distance_accuracy(ocr_text, ground_truth):
    """Computes normalized Levenshtein-based accuracy"""
    distance = Levenshtein.distance(ocr_text, ground_truth)
    max_len = max(len(ocr_text), len(ground_truth))
    return (1 - distance / max_len) if max_len > 0 else 0
    
print("Reset variables")
correct_characters = 0
invalid_images = 0

# Process each image
#for index, row in tqdm(df_google.iterrows(), total=len(df_google)):
for index, row in tqdm(df_google.iterrows(), total=10):

    image_path = row["image_path"]
    ground_truth = row["plate_number"]
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Error: Unable to load image at {image_path}")
        invalid_images += 1
        continue
        
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    #show the original image
    plotted_img = image
    plt.figure(figsize=(8, 6))
    plt.imshow(plotted_img)
    plt.axis('off')
    plt.show()

    results = model.predict(image, imgsz=640)

    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        class_id = int(result.cls[0])
        label = classes[class_id]

        #get the license plate
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        #cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (152, 215, 235), 2)

        #show the image with the license plate
        plotted_img = image
        plt.figure(figsize=(8, 6))
        plt.imshow(plotted_img)
        plt.axis('off')
        plt.show()
            
       # Ensure bounding box is within image boundaries
        h, w, _ = image.shape
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

        # Crop the license plate
        cropped_plate = image[y1:y2, x1:x2]
        
       # Display cropped license plate using Matplotlib
        plt.figure(figsize=(4, 2))
        plt.imshow(cv2.cvtColor(cropped_plate, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for Matplotlib
        plt.axis("off")
        plt.title("Cropped License Plate")
        plt.show()
   
        # Initialize EasyOCR reader
        
        #reader = easyocr.Reader(['en'])

        # Run OCR on cropped plate
        #ocr_text = reader.readtext(cropped_plate, detail=0)

        logging.getLogger("ppocr").setLevel(logging.ERROR)  # Suppress PaddleOCR debug logs
        ocr = PaddleOCR(use_angle_cls=True, lang="en")

        # Run OCR
        ocr_result = ocr.ocr(cropped_plate, cls=True)

        # Extract license plate text
        license_plate_text = extract_license_plate(ocr_result)

        #license_plate_text = ''.join(ocr_text).replace(" ", "")

        
        print("Detected license_plate_text ",license_plate_text)
        print("ground_truth",ground_truth)

        # Overlay detected text on the bounding box
        cv2.putText(image, license_plate_text, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show the final image with text overlay
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Detected License Plate with OCR Text")
        plt.show()
    
        # Character-level accuracy
        #char_accuracy = character_level_accuracy(ocr_result, ground_truth) * 100
        #character_accuracy_sum += char_accuracy
        #print(f"Character-level accuracy: {char_accuracy:.2f}%")
        #print(f"Running Character-level accuracy: {character_accuracy_sum:.2f}%")
        
        # Word-level accuracy
        word_accuracy = word_level_accuracy(license_plate_text, ground_truth) * 100
        print(f"Word-level accuracy: {word_accuracy}%")
        if word_accuracy == 100:
            correct_predictions += 1

        lev_accuracy = levenshtein_distance_accuracy(license_plate_text, ground_truth) * 100
        levenshtein_accuracy_sum += lev_accuracy
        
        # Store result
        #detected_plates.append(" ".join(ocr_result))

        # Print detected license plates
        print("correct_predictions so far:", correct_predictions)
        print("levenshtein_accuracy_sum:", levenshtein_accuracy_sum)

# Calculate final accuracy scores
valid_images = len(df_google)-invalid_images
word_level_accuracy = (correct_predictions / valid_images) * 100
average_levenshtein_accuracy = (levenshtein_accuracy_sum / valid_images)

#character_level_accuracy = (character_accuracy_sum / len(test_df)) 

print(f"✅ Word-Level Accuracy: {word_level_accuracy:.2f}%")
print(f"✅ Levenshtein-Based Accuracy: {average_levenshtein_accuracy:.2f}%")
print(f"invalid images :",invalid_images)
print(f"valid images :",valid_images)
#print(f"✅ Character-Level Accuracy: {character_level_accuracy:.2f}%")

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import cv2
import os
import easyocr
from difflib import SequenceMatcher
from tqdm import tqdm
import logging
import Levenshtein

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

classes = ["Licence Plate"]

# List to store OCR results
detected_plates = []

# Accuracy counters
correct_predictions = 0
total_characters = 0
character_accuracy_sum = 0
levenshtein_accuracy_sum = 0

# Function to calculate character-level accuracy
def character_level_accuracy(ocr_text, ground_truth):
    ocr_text = ''.join(ocr_text).replace(" ", "").lower()
    print("Final OCR Text:", ocr_text)
    ground_truth = ground_truth.replace(" ", "").lower()
    
    correct_characters = sum(1 for i in range(min(len(ocr_text), len(ground_truth))) if ocr_text[i] == ground_truth[i])
    
    return correct_characters / len(ground_truth) if len(ground_truth) > 0 else 0

# Function to calculate word-level accuracy (Exact match)
def word_level_accuracy(ocr_text, ground_truth):
    return 1 if ocr_text.strip().upper() == ground_truth.strip().upper() else 0

# Function to calculate Levenshtein Distance-based accuracy
def levenshtein_distance_accuracy(ocr_text, ground_truth):
    distance = Levenshtein.distance(ocr_text, ground_truth)
    max_len = max(len(ocr_text), len(ground_truth))
    return (1 - distance / max_len) if max_len > 0 else 0

# Function to extract license plate text using EasyOCR
def extract_license_plate_easyocr(ocr_result):
    """
    Custom function to extract license plate text from EasyOCR results.
    Modify this function to filter noise and improve recognition accuracy.
    """
    extracted_text = []
    for detection in ocr_result:
        text = detection[1]  # Assuming EasyOCR returns (bbox, text, confidence)
        extracted_text.append(text)
    
    return ''.join(extracted_text).replace(" ", "").upper()

print("Reset variables")
correct_characters = 0
invalid_images = 0

# Process each image
for index, row in tqdm(df_google.iterrows(), total=len(df_google)):
    image_path = row["image_path"]
    ground_truth = row["plate_number"]
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Error: Unable to load image at {image_path}")
        invalid_images += 1
        continue
        
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Show original image
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    results = model.predict(image, imgsz=640)

    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        class_id = int(result.cls[0])
        label = classes[class_id]

        # Draw bounding box around license plate
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Show image with bounding box
        plt.figure(figsize=(8, 6))
        plt.imshow(image)
        plt.axis('off')
        plt.show()
            
        # Ensure bounding box is within image boundaries
        h, w, _ = image.shape
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

        # Crop the license plate
        cropped_plate = image[y1:y2, x1:x2]
        
        # Display cropped license plate
        plt.figure(figsize=(4, 2))
        plt.imshow(cv2.cvtColor(cropped_plate, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Cropped License Plate")
        plt.show()
   
        # Run OCR on cropped plate using EasyOCR
        ocr_result = reader.readtext(cropped_plate, detail=1)  # Get full OCR details

        # Extract license plate text using the custom function
        license_plate_text = extract_license_plate_easyocr(ocr_result)

        print("Detected License Plate:", license_plate_text)
        print("Ground Truth:", ground_truth)

        # Overlay detected text on the bounding box
        cv2.putText(image, license_plate_text, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show final image with text overlay
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Detected License Plate with OCR Text")
        plt.show()
    
        # Word-level accuracy
        word_accuracy = word_level_accuracy(license_plate_text, ground_truth) * 100
        print(f"Word-Level Accuracy: {word_accuracy}%")
        if word_accuracy == 100:
            correct_predictions += 1

        lev_accuracy = levenshtein_distance_accuracy(license_plate_text, ground_truth) * 100
        levenshtein_accuracy_sum += lev_accuracy

        print("Correct Predictions so far:", correct_predictions)
        print("Levenshtein Accuracy Sum:", levenshtein_accuracy_sum)

# Calculate final accuracy scores
valid_images = len(df_google) - invalid_images
word_level_accuracy = (correct_predictions / valid_images) * 100
average_levenshtein_accuracy = (levenshtein_accuracy_sum / valid_images)

print(f"✅ Word-Level Accuracy: {word_level_accuracy:.2f}%")
print(f"✅ Levenshtein-Based Accuracy: {average_levenshtein_accuracy:.2f}%")
print(f"Invalid Images:", invalid_images)
print(f"Valid Images:", valid_images)
